In [1]:
#Importing the libraries

import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from imutils import paths
import pickel

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D , Dropout , Flatten , Dense , Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator , img_to_array , load_img
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [2]:
dataset = r'C:\Users\Ritika Singh\Desktop\Corona Virus Mask Detection\dataset'
imagePaths = list(paths.list_images(dataset))

In [3]:
#  imagePaths

In [4]:
# We are specifying the label values
labels =[]
data=[]
for i in imagePaths:
    label = i.split(os.path.sep)[-2]
    labels.append(label)
    # Converting the images into list of numbers to feed into our model for training
    image = load_img(i , target_size=(224,224))#Extrating the image and since all the size of images are different we are converting them all into a standard  size of (224x224)
    image = img_to_array(image)#Image is convrted into list of numbers(float_values)
    image = preprocess_input(image)# This reduces the size of the input (scaled - down)
    data.append(image)


C:\ana\lib\site-packages\PIL\Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [5]:
# labels

In [6]:
# image

In [7]:
# Converting the list  into numpy array 
data = np.array(data,dtype='float32')
labels = np.array(labels)

In [8]:
data #4d array

array([[[[-0.94509804, -0.9764706 , -0.85882354],
         [-0.94509804, -0.9764706 , -0.85882354],
         [-0.94509804, -0.9764706 , -0.85882354],
         ...,
         [-0.8509804 , -0.90588236, -0.78039217],
         [-0.8509804 , -0.90588236, -0.78039217],
         [-0.84313726, -0.8980392 , -0.77254903]],

        [[-0.9137255 , -0.94509804, -0.827451  ],
         [-0.9137255 , -0.94509804, -0.827451  ],
         [-0.9137255 , -0.94509804, -0.827451  ],
         ...,
         [-0.7647059 , -0.79607844, -0.6784314 ],
         [-0.7647059 , -0.79607844, -0.6784314 ],
         [-0.75686276, -0.7882353 , -0.67058825]],

        [[-0.92156863, -0.9607843 , -0.8039216 ],
         [-0.92156863, -0.9607843 , -0.8039216 ],
         [-0.92156863, -0.9607843 , -0.8039216 ],
         ...,
         [-0.88235295, -0.8980392 , -0.7882353 ],
         [-0.8666667 , -0.88235295, -0.77254903],
         [-0.85882354, -0.8745098 , -0.7647059 ]],

        ...,

        [[ 0.3803922 , -0.9607843 , -0

In [9]:
data.shape

(3833, 224, 224, 3)

In [10]:
labels

array(['without_mask', 'without_mask', 'without_mask', ..., 'with_mask',
       'with_mask', 'with_mask'], dtype='<U12')

In [11]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)# converting into categorical data

In [12]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [13]:
x_train ,x_test ,y_train , y_test = train_test_split(data, labels, test_size = 0.2, random_state = 10 , stratify = labels)


In [14]:
print(x_train.shape , y_train.shape)

(3066, 224, 224, 3) (3066, 2)


In [15]:
print(x_test.shape , y_test.shape)

(767, 224, 224, 3) (767, 2)


In [16]:
# Augumentation

aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.20,
    shear_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [17]:
aug

In [18]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape= (224,224,3)))

In [19]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [20]:
headModel = base_model.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name = "Flatten")(headModel)
headModel = Dense(128 , activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2 , activation="softmax")(headModel)
model = Model(inputs  = base_model.input , outputs = headModel)

In [21]:
for layer in base_model.layers:
    layer.trainable = False

In [22]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [25]:
learning_rate = 0.001
Epochs = 20
Batch_size  = 12
optimizer = Adam(learning_rate=learning_rate , decay = learning_rate/Epochs)
model.compile(loss = 'binary_crossentropy' , optimizer=optimizer , metrics=['accuracy'])
history = model.fit(
    aug.flow(x_train ,y_train , batch_size =Batch_size ),
    steps_per_epoch= len(x_train)//Batch_size,
    validation_data= (x_test , y_test),
    validation_steps= len(x_test)//Batch_size,
    epochs=Epochs    
)

model.save(r'C:\Users\Ritika Singh\Desktop\Corona Virus Mask Detection\my_weights.h5')

Epoch 1/20
255/255 [==============================] - 61s 238ms/step - loss: 0.1217 - accuracy: 0.9558 - val_loss: 0.0670 - val_accuracy: 0.9791
Epoch 2/20
255/255 [==============================] - 62s 244ms/step - loss: 0.0982 - accuracy: 0.9614
Epoch 3/20
255/255 [==============================] - 60s 234ms/step - loss: 0.0753 - accuracy: 0.9745
Epoch 4/20
255/255 [==============================] - 62s 242ms/step - loss: 0.0734 - accuracy: 0.9725
Epoch 5/20
255/255 [==============================] - 62s 245ms/step - loss: 0.0584 - accuracy: 0.9810
Epoch 6/20
255/255 [==============================] - 62s 242ms/step - loss: 0.0571 - accuracy: 0.9813
Epoch 7/20
255/255 [==============================] - 61s 241ms/step - loss: 0.0606 - accuracy: 0.9751
Epoch 8/20
255/255 [==============================] - 62s 242ms/step - loss: 0.0537 - accuracy: 0.9817
Epoch 9/20
255/255 [==============================] - 62s 243ms/step - loss: 0.0528 - accuracy: 0.9777
Epoch 10/20
255/255 [==========

In [26]:
predict = model.predict(x_test, batch_size=Batch_size)
predict  = np.argmax(predict , axis=1)
print(classification_report(y_test.argmax(axis = 1) , predict , target_names =lb.classes_))


              precision    recall  f1-score   support

   with_mask       0.98      1.00      0.99       383
without_mask       1.00      0.98      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767

